# Variational AutoEncoder (MLP) Pytorch
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/pytorch/examples/blob/master/mnist/main.py
* https://github.com/pytorch/examples/blob/master/vae/main.py
* http://wiseodd.github.io/techblog/2017/01/24/vae-pytorch/
* https://github.com/ritheshkumar95/pytorch-vqvae

In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.distributions.normal import Normal
from torchvision import datasets, transforms
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter
batch_size = 128
ZDIMS = 20
epochs = 10
print("PyTorch Version: ",torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
num_gpu = torch.cuda.device_count()
print('Number of available GPUs:', num_gpu)

log_interval = 10

PyTorch Version:  1.0.0
Device: cpu
Number of available GPUs: 0


#### Download Data

In [2]:
# Download or load downloaded MNIST dataset
# shuffle data at every epoch
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', 
    train=True, download=True,transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

# Same for test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

#### Define Model

In [3]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, ZDIMS)
        self.fc22 = nn.Linear(400, ZDIMS)
        self.fc3 = nn.Linear(ZDIMS, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [4]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#### Define Loss Function

In [5]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    #KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    #KLD = F.kl_div(, reduction='sum')
    q_z_x = Normal(mu, logvar.mul(.5).exp())
    p_z = Normal(torch.zeros_like(mu), torch.ones_like(logvar))
    KLD = F.kl_div(q_z_x.sample(), p_z.sample(), reduction='sum')

    return BCE + KLD

#### Define Train/Test Functions

In [6]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

#### Train/Test

In [7]:
for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 546.737305
Train Epoch: 1 [1280/60000 (2%)]	Loss: 281.442078
Train Epoch: 1 [2560/60000 (4%)]	Loss: 230.110092
Train Epoch: 1 [3840/60000 (6%)]	Loss: 222.175140
Train Epoch: 1 [5120/60000 (9%)]	Loss: 216.356171
Train Epoch: 1 [6400/60000 (11%)]	Loss: 199.507782
Train Epoch: 1 [7680/60000 (13%)]	Loss: 199.690155
Train Epoch: 1 [8960/60000 (15%)]	Loss: 194.598907
Train Epoch: 1 [10240/60000 (17%)]	Loss: 182.367508
Train Epoch: 1 [11520/60000 (19%)]	Loss: 169.917160
Train Epoch: 1 [12800/60000 (21%)]	Loss: 156.974976
Train Epoch: 1 [14080/60000 (23%)]	Loss: 155.357376
Train Epoch: 1 [15360/60000 (26%)]	Loss: 153.320587
Train Epoch: 1 [16640/60000 (28%)]	Loss: 145.187180
Train Epoch: 1 [17920/60000 (30%)]	Loss: 143.542450
Train Epoch: 1 [19200/60000 (32%)]	Loss: 137.570343
Train Epoch: 1 [20480/60000 (34%)]	Loss: 134.114029
Train Epoch: 1 [21760/60000 (36%)]	Loss: 131.512268
Train Epoch: 1 [23040/60000 (38%)]	Loss: 133.719025
Train Epoch: 1 [24320/60000 

Train Epoch: 4 [21760/60000 (36%)]	Loss: 82.059212
Train Epoch: 4 [23040/60000 (38%)]	Loss: 76.172699
Train Epoch: 4 [24320/60000 (41%)]	Loss: 81.782173
Train Epoch: 4 [25600/60000 (43%)]	Loss: 79.720245
Train Epoch: 4 [26880/60000 (45%)]	Loss: 81.274490
Train Epoch: 4 [28160/60000 (47%)]	Loss: 75.113358
Train Epoch: 4 [29440/60000 (49%)]	Loss: 77.498711
Train Epoch: 4 [30720/60000 (51%)]	Loss: 79.058083
Train Epoch: 4 [32000/60000 (53%)]	Loss: 78.894814
Train Epoch: 4 [33280/60000 (55%)]	Loss: 78.121521
Train Epoch: 4 [34560/60000 (58%)]	Loss: 79.920830
Train Epoch: 4 [35840/60000 (60%)]	Loss: 79.650696
Train Epoch: 4 [37120/60000 (62%)]	Loss: 79.326202
Train Epoch: 4 [38400/60000 (64%)]	Loss: 79.028809
Train Epoch: 4 [39680/60000 (66%)]	Loss: 80.408127
Train Epoch: 4 [40960/60000 (68%)]	Loss: 79.723770
Train Epoch: 4 [42240/60000 (70%)]	Loss: 76.816666
Train Epoch: 4 [43520/60000 (72%)]	Loss: 80.273972
Train Epoch: 4 [44800/60000 (75%)]	Loss: 81.538872
Train Epoch: 4 [46080/60000 (77

Train Epoch: 7 [43520/60000 (72%)]	Loss: 79.226158
Train Epoch: 7 [44800/60000 (75%)]	Loss: 74.408127
Train Epoch: 7 [46080/60000 (77%)]	Loss: 77.085251
Train Epoch: 7 [47360/60000 (79%)]	Loss: 73.054314
Train Epoch: 7 [48640/60000 (81%)]	Loss: 74.193298
Train Epoch: 7 [49920/60000 (83%)]	Loss: 76.868210
Train Epoch: 7 [51200/60000 (85%)]	Loss: 78.098686
Train Epoch: 7 [52480/60000 (87%)]	Loss: 73.331566
Train Epoch: 7 [53760/60000 (90%)]	Loss: 73.067947
Train Epoch: 7 [55040/60000 (92%)]	Loss: 71.856728
Train Epoch: 7 [56320/60000 (94%)]	Loss: 71.465126
Train Epoch: 7 [57600/60000 (96%)]	Loss: 74.013885
Train Epoch: 7 [58880/60000 (98%)]	Loss: 78.448105
====> Epoch: 7 Average loss: 75.7419
====> Test set loss: 75.5134
Train Epoch: 8 [0/60000 (0%)]	Loss: 76.042870
Train Epoch: 8 [1280/60000 (2%)]	Loss: 76.125610
Train Epoch: 8 [2560/60000 (4%)]	Loss: 74.643623
Train Epoch: 8 [3840/60000 (6%)]	Loss: 74.557060
Train Epoch: 8 [5120/60000 (9%)]	Loss: 72.017883
Train Epoch: 8 [6400/60000 (1